#### **LEVEL - 1**

In [ ]:
import pandas as pd

train_df = pd.read_csv('UNSW_NB15_training-set.csv')
test_df = pd.read_csv('UNSW_NB15_testing-set.csv')

train_df.head(), test_df.head()

(   id       dur proto service state  spkts  dpkts  sbytes  dbytes       rate  \
 0   1  0.121478   tcp       -   FIN      6      4     258     172  74.087490   
 1   2  0.649902   tcp       -   FIN     14     38     734   42014  78.473372   
 2   3  1.623129   tcp       -   FIN      8     16     364   13186  14.170161   
 3   4  1.681642   tcp     ftp   FIN     12     12     628     770  13.677108   
 4   5  0.449454   tcp       -   FIN     10      6     534     268  33.373826   
 
    ...  ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  ct_ftp_cmd  \
 0  ...                 1               1             0           0   
 1  ...                 1               2             0           0   
 2  ...                 1               3             0           0   
 3  ...                 1               3             1           1   
 4  ...                 1              40             0           0   
 
    ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  is_sm_ips_ports  attack_cat  \
 0     

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

train = train_df.copy()
test = test_df.copy()

train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

cat_cols = ['proto', 'service', 'state', 'attack_cat']

for col in cat_cols:
    le = LabelEncoder()
    all_vals = pd.concat([train[col], test[col]], axis=0)
    le.fit(all_vals)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

X_train = train.drop(columns=['label'])
y_train = train['label']

X_test = test.drop(columns=['label'])
y_test = test['label']

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Preprocessing done without unseen errors.")

Preprocessing done without unseen errors.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,929 (19.25 KB)

 Trainable params: 4,929 (19.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=15,
    batch_size=256,
    verbose=1
)

Epoch 1/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9264 - loss: 0.1899 - val_accuracy: 0.9967 - val_loss: 0.0130
Epoch 2/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9954 - loss: 0.0183 - val_accuracy: 0.9993 - val_loss: 0.0041
Epoch 3/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9993 - loss: 0.0035 - val_accuracy: 0.9996 - val_loss: 0.0019
Epoch 4/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9998 - loss: 0.0019 - val_accuracy: 0.9999 - val_loss: 8.2992e-04
Epoch 5/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9999 - loss: 0.0010 - val_accuracy: 0.9999 - val_loss: 5.5760e-04
Epoch 6/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9999 - loss: 4.0961e-04 - val_accuracy: 0.9998 - val_loss: 5.0922e-04
Epoch 7/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 2.5310e-04 - val_accuracy: 0.9998 - val_loss: 7.4202e-04
Epoch 8/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 1.0000 - loss: 1

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Basic accuracy
test_acc = (y_pred.flatten() == y_test).mean()
print(f"Test Accuracy: {test_acc:.4f}")

# F1 / Precision / Recall
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


2573/2573 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Test Accuracy: 0.9997

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37000
           1       1.00      1.00      1.00     45332

    accuracy                           1.00     82332
   macro avg       1.00      1.00      1.00     82332
weighted avg       1.00      1.00      1.00     82332


Confusion Matrix:
[[36989    11]
 [   11 45321]]


In [ ]:
# Save model
model.save("netpulse_level1.h5")

# Save scaler
import pickle
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model + Scaler saved for Level 1.")

Model + Scaler saved for Level 1.


In [ ]:
# Pick one random sample from test
import numpy as np

idx = np.random.randint(0, len(X_test))
sample = X_test[idx:idx+1]

pred = model.predict(sample)[0][0]

label = "ATTACK" if pred > 0.5 else "NORMAL"

print(f"Sample #{idx} → {label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Sample #34360 → NORMAL


In [ ]:
for i in range(10):
    idx = np.random.randint(0, len(X_test))
    sample = X_test[idx:idx+1]
    pred = model.predict(sample)[0][0]
    label = "ATTACK" if pred > 0.5 else "NORMAL"
    print(f"{i+1}. #{idx} → {label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1. #9737 → ATTACK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2. #5337 → ATTACK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
3. #65931 → NORMAL
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
4. #66986 → NORMAL
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
5. #80235 → NORMAL
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
6. #16989 → ATTACK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
7. #18590 → ATTACK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
8. #14009 → ATTACK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
9. #13186 → ATTACK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
10. #794 → ATTACK


In [ ]:
print("=== NET PULSE — LEVEL 1 SUMMARY ===")
print("Dataset     : UNSW-NB15")
print("Mode        : Binary Intrusion Detection (NORMAL vs ATTACK)")
print("Model       : Feedforward Neural Network (Dense 64 → 32 → 1)")
print("Labels      : Normal (0), Attack (1)")
print("Train Size  :", len(X_train))
print("Test Size   :", len(X_test))
print("Test Acc    : {:.4f}".format(test_acc))
print("Status      : LEVEL 1 COMPLETE ✔")
print("Artifacts   : netpulse_level1.h5 + scaler.pkl")
print("Ready For   : Level 2 (Flow sequencing + CNN+LSTM + Multi-class + UI)")
print("====================================")

=== NET PULSE — LEVEL 1 SUMMARY ===
Dataset     : UNSW-NB15
Mode        : Binary Intrusion Detection (NORMAL vs ATTACK)
Model       : Feedforward Neural Network (Dense 64 → 32 → 1)
Labels      : Normal (0), Attack (1)
Train Size  : 175341
Test Size   : 82332
Test Acc    : 0.9997
Status      : LEVEL 1 COMPLETE ✔
Artifacts   : netpulse_level1.h5 + scaler.pkl
Ready For   : Level 2 (Flow sequencing + CNN+LSTM + Multi-class + UI)


# **LEVEL - 2**

In [ ]:
# Make a copy to avoid mutation
train2 = train_df.copy()
test2 = test_df.copy()

# Define simplified mapping
low_suspicious = ['Reconnaissance', 'Fuzzers', 'Analysis']
high_malicious = ['DoS', 'Exploits', 'Shellcode', 'Worms', 'Generic', 'Backdoor']

def map_attack(cat):
    if cat == 'Normal':
        return 0
    elif cat in low_suspicious:
        return 1
    elif cat in high_malicious:
        return 2
    else:
        return 2  # unknown -> treat as high severity for safety

train2['class3'] = train2['attack_cat'].apply(map_attack)
test2['class3'] = test2['attack_cat'].apply(map_attack)

# Quick sanity check
print(train2['class3'].value_counts())
print(test2['class3'].value_counts())


class3
2    88666
0    56000
1    30675
Name: count, dtype: int64
class3
0    37000
2    35097
1    10235
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import StandardScaler

# Drop id
train3 = train2.drop(columns=['id'])
test3 = test2.drop(columns=['id'])

# Same categorical columns
cat_cols = ['proto', 'service', 'state']

# Encode categories (train + test combo)
from sklearn.preprocessing import LabelEncoder

for col in cat_cols:
    le = LabelEncoder()
    all_vals = pd.concat([train3[col], test3[col]], axis=0)
    le.fit(all_vals)
    train3[col] = le.transform(train3[col])
    test3[col] = le.transform(test3[col])

# Features & labels
X_train3 = train3.drop(columns=['label', 'attack_cat', 'class3'])
y_train3 = train3['class3']

X_test3 = test3.drop(columns=['label', 'attack_cat', 'class3'])
y_test3 = test3['class3']

# Scale
scaler3 = StandardScaler()
X_train3 = scaler3.fit_transform(X_train3)
X_test3 = scaler3.transform(X_test3)

print("Multiclass preprocessing done.")

Multiclass preprocessing done.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# One-hot encode labels
y_train3_cat = to_categorical(y_train3, num_classes=3)
y_test3_cat = to_categorical(y_test3, num_classes=3)

# Baseline model
mc_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train3.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

mc_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

mc_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │         2,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,931 (19.26 KB)

 Trainable params: 4,931 (19.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
mc_history = mc_model.fit(
    X_train3, y_train3_cat,
    validation_split=0.2,
    epochs=15,
    batch_size=256,
    verbose=1
)

Epoch 1/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7695 - loss: 0.5488 - val_accuracy: 0.9503 - val_loss: 0.1494
Epoch 2/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8471 - loss: 0.3567 - val_accuracy: 0.9486 - val_loss: 0.1415
Epoch 3/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8532 - loss: 0.3388 - val_accuracy: 0.9491 - val_loss: 0.1360
Epoch 4/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8595 - loss: 0.3249 - val_accuracy: 0.9529 - val_loss: 0.1282
Epoch 5/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8634 - loss: 0.3194 - val_accuracy: 0.9560 - val_loss: 0.1219
Epoch 6/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8658 - loss: 0.3117 - val_accuracy: 0.9561 - val_loss: 0.1215
Epoch 7/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8679 - loss: 0.3096 - val_accuracy: 0.9514 - val_loss: 0.1299
Epoch 8/15
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8693 - loss: 0.3062 - val_accuracy: 0.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

pred_mc = mc_model.predict(X_test3)
pred_mc = pred_mc.argmax(axis=1)

print("Test Accuracy:", (pred_mc == y_test3).mean())
print("\nReport:\n", classification_report(y_test3, pred_mc))
print("\nConfusion Matrix:\n", confusion_matrix(y_test3, pred_mc))

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Test Accuracy: 0.7989481611038235

Report:
               precision    recall  f1-score   support

           0       0.94      0.72      0.81     37000
           1       0.38      0.64      0.47     10235
           2       0.89      0.93      0.91     35097

    accuracy                           0.80     82332
   macro avg       0.74      0.76      0.73     82332
weighted avg       0.85      0.80      0.81     82332


Confusion Matrix:
 [[26511  9252  1237]
 [  844  6524  2867]
 [  789  1564 32744]]


In [ ]:
import numpy as np

SEQ_LEN = 50

def flow_to_sequence(row):
    spkts = int(row['spkts'])
    dpkts = int(row['dpkts'])
    dur = float(row['dur']) if row['dur'] > 0 else 0.001  # prevent div by zero

    # sizes
    sbytes = row['sbytes']
    dbytes = row['dbytes']

    # packet size approx
    if spkts > 0:
        src_sizes = [sbytes/spkts]*spkts
    else:
        src_sizes = []

    if dpkts > 0:
        dst_sizes = [dbytes/dpkts]*dpkts
    else:
        dst_sizes = []

    # timestamps (uniform distribution for simplicity)
    src_times = np.linspace(0, dur, len(src_sizes)) if len(src_sizes)>0 else []
    dst_times = np.linspace(0, dur, len(dst_sizes)) if len(dst_sizes)>0 else []

    # direction tagging
    src_dir = [+1]*len(src_sizes)
    dst_dir = [-1]*len(dst_sizes)

    # merge in time order
    packets = list(zip(src_times, src_sizes, src_dir)) + \
              list(zip(dst_times, dst_sizes, dst_dir))

    packets.sort(key=lambda x: x[0])  # sort by time

    # time gaps
    seq = []
    prev_t = 0
    for t, size, d in packets:
        gap = t - prev_t
        prev_t = t
        seq.append([size, gap, d])

    # pad / truncate to SEQ_LEN
    if len(seq) < SEQ_LEN:
        pad = [[0,0,0]]*(SEQ_LEN - len(seq))
        seq = seq + pad
    else:
        seq = seq[:SEQ_LEN]

    return seq


In [ ]:
# IMPORTANT: use train3 & test3 from multi-class preprocessing

train_sequences = np.array([flow_to_sequence(row) for _, row in train3.iterrows()])
test_sequences = np.array([flow_to_sequence(row) for _, row in test3.iterrows()])

print("Sequence shapes:")
print(train_sequences.shape, test_sequences.shape)

Sequence shapes:
(175341, 50, 3) (82332, 50, 3)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, BatchNormalization

# Build CNN + LSTM Hybrid
cnn_lstm = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(SEQ_LEN, 3)),
    BatchNormalization(),
    LSTM(32, return_sequences=False),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

cnn_lstm.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_lstm.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 48, 32)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,923 (38.76 KB)

 Trainable params: 9,859 (38.51 KB)

 Non-trainable params: 64 (256.00 B)

In [ ]:
hist_seq = cnn_lstm.fit(
    train_sequences, y_train3,
    validation_split=0.15,
    epochs=10,
    batch_size=256,
    verbose=1
)

Epoch 1/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.5737 - loss: 0.9026 - val_accuracy: 0.8928 - val_loss: 0.4499
Epoch 2/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 42s 64ms/step - accuracy: 0.7284 - loss: 0.6715 - val_accuracy: 0.9237 - val_loss: 0.4249
Epoch 3/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 37s 64ms/step - accuracy: 0.7525 - loss: 0.6281 - val_accuracy: 0.9300 - val_loss: 0.4239
Epoch 4/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 37s 64ms/step - accuracy: 0.7520 - loss: 0.6136 - val_accuracy: 0.9335 - val_loss: 0.4004
Epoch 5/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 38s 64ms/step - accuracy: 0.7613 - loss: 0.5924 - val_accuracy: 0.9335 - val_loss: 0.4124
Epoch 6/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 37s 63ms/step - accuracy: 0.7615 - loss: 0.5890 - val_accuracy: 0.9314 - val_loss: 0.4104
Epoch 7/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 40s 68ms/step - accuracy: 0.7536 - loss: 0.6013 - val_accuracy: 0.9343 - val_loss: 0.4015
Epoch 8/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 41s 68ms/step - accuracy: 0.7644 - loss: 0.5781 - 

In [ ]:
pred_seq = cnn_lstm.predict(test_sequences)
pred_seq = pred_seq.argmax(axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print("Test Accuracy:", (pred_seq == y_test3).mean())
print("\nReport:\n", classification_report(y_test3, pred_seq))
print("\nConfusion Matrix:\n", confusion_matrix(y_test3, pred_seq))


2573/2573 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step
Test Accuracy: 0.7198659087596561

Report:
               precision    recall  f1-score   support

           0       0.96      0.57      0.72     37000
           1       0.32      0.49      0.39     10235
           2       0.74      0.94      0.83     35097

    accuracy                           0.72     82332
   macro avg       0.67      0.67      0.65     82332
weighted avg       0.79      0.72      0.72     82332


Confusion Matrix:
 [[21221  9039  6740]
 [  241  5038  4956]
 [  570  1518 33009]]


In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv1D, LSTM, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model

# Input A: Flow stats
flow_input = Input(shape=(X_train3.shape[1],))
flow_branch = Dense(32, activation='relu')(flow_input)
flow_branch = BatchNormalization()(flow_branch)

# Input B: Sequence
seq_input = Input(shape=(SEQ_LEN, 3))
seq_branch = Conv1D(32, kernel_size=3, activation='relu')(seq_input)
seq_branch = BatchNormalization()(seq_branch)
seq_branch = LSTM(32)(seq_branch)

# Merge branches
merged = Concatenate()([flow_branch, seq_branch])
merged = Dense(32, activation='relu')(merged)
merged = Dropout(0.2)(merged)
output = Dense(3, activation='softmax')(merged)

fusion_model = Model(inputs=[flow_input, seq_input], outputs=output)

fusion_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

fusion_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 50, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 42)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 48, 32)    │        320 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      1,376 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 48, 32)    │        128 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32)        │        128 │ dense_8[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32)        │      8,320 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32)        │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 3)         │         99 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,451 (48.64 KB)

 Trainable params: 12,323 (48.14 KB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
hist_fusion = fusion_model.fit(
    [X_train3, train_sequences],
    y_train3,
    validation_split=0.15,
    epochs=10,
    batch_size=256,
    verbose=1
)

Epoch 1/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 45s 70ms/step - accuracy: 0.7181 - loss: 0.6256 - val_accuracy: 0.9472 - val_loss: 0.1382
Epoch 2/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 38s 65ms/step - accuracy: 0.8360 - loss: 0.3744 - val_accuracy: 0.9540 - val_loss: 0.1292
Epoch 3/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 47s 75ms/step - accuracy: 0.8512 - loss: 0.3449 - val_accuracy: 0.9574 - val_loss: 0.1221
Epoch 4/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 40s 68ms/step - accuracy: 0.8606 - loss: 0.3303 - val_accuracy: 0.9538 - val_loss: 0.1224
Epoch 5/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 40s 66ms/step - accuracy: 0.8641 - loss: 0.3215 - val_accuracy: 0.9569 - val_loss: 0.1204
Epoch 6/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.8681 - loss: 0.3126 - val_accuracy: 0.9604 - val_loss: 0.1097
Epoch 7/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 38s 65ms/step - accuracy: 0.8693 - loss: 0.3098 - val_accuracy: 0.9568 - val_loss: 0.1154
Epoch 8/10
583/583 ━━━━━━━━━━━━━━━━━━━━ 38s 65ms/step - accuracy: 0.8717 - loss: 0.3045 - 

In [ ]:
pred_fusion = fusion_model.predict([X_test3, test_sequences])
pred_fusion = pred_fusion.argmax(axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print("Test Accuracy:", (pred_fusion == y_test3).mean())
print("\nReport:\n", classification_report(y_test3, pred_fusion))
print("\nConfusion Matrix:\n", confusion_matrix(y_test3, pred_fusion))

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step
Test Accuracy: 0.808033328474955

Report:
               precision    recall  f1-score   support

           0       0.95      0.75      0.84     37000
           1       0.39      0.61      0.48     10235
           2       0.88      0.93      0.90     35097

    accuracy                           0.81     82332
   macro avg       0.74      0.76      0.74     82332
weighted avg       0.85      0.81      0.82     82332


Confusion Matrix:
 [[27681  7590  1729]
 [ 1216  6224  2795]
 [  318  2157 32622]]


In [ ]:
# Save fusion model
fusion_model.save("netpulse_fusion_v2.h5")
print("Fusion model saved as netpulse_fusion_v2.h5")

Fusion model saved as netpulse_fusion_v2.h5


In [ ]:
import pickle

with open("scaler3.pkl", "wb") as f:
    pickle.dump(scaler3, f)

print("Flow scaler saved as scaler3.pkl")

Flow scaler saved as scaler3.pkl


In [ ]:
class_map = {0: "Normal", 1: "Suspicious", 2: "Malicious"}

with open("class_map.pkl", "wb") as f:
    pickle.dump(class_map, f)

print("Class map saved as class_map.pkl")

Class map saved as class_map.pkl


In [ ]:
config = {
    "SEQ_LEN": SEQ_LEN,
    "version": "v2_fusion",
    "features": ["size", "time_gap", "direction"],
}

with open("config.pkl","wb") as f:
    pickle.dump(config, f)

print("Config saved as config.pkl")

Config saved as config.pkl


# **APP**

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb

--2026-01-23 05:40:16--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.1.1/cloudflared-linux-amd64.deb [following]
--2026-01-23 05:40:16--  https://github.com/cloudflare/cloudflared/releases/download/2026.1.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/12aae843-3db0-4414-9b56-e2a442db3e76?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-23T06%3A21%3A40Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4d

In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 & sleep 3 && cloudflared tunnel --url http://localhost:8501

/bin/bash: line 1: streamlit: command not found
2026-01-23T05:40:23Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-23T05:40:23Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-23T05:40:27Z INF +--------------------------------------------------------------------------------------------+
2026-01-23T05:40:27Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-01-23T05:40:27Z INF |  https